# Covid-19 Dashboard

In [ ]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [ ]:
# making figures larger
%matplotlib inline
plt.rcParams['figure.dpi'] = 100

## Loading initial data from disk


In [ ]:
# first implementation 
#importing necessary libraries 
from uk_covid19 import Cov19API
import json 


In [ ]:
#determining filter - parameters, in this case, areaType parameter and areaName
England = [ 
    "areaType=Nation", 
    "areaName=England" 
]


# defining a structure and specifying data fields requested according to SDK documentation and API documentation

AdmissionAndCasesComparison = { 
    "date": "date",
    "New Admissions": "newAdmissions",
     "New Cases":  "newCasesByPublishDate", 
   "Hospital Cases" : "hospitalCases", 
  
}


# creating Cov19API object to access API 
# passing filters into it with a structure placed to its constructor

AdmissionAndCases = Cov19API(filters=England, structure=AdmissionAndCasesComparison)


# retrieving the repsonse from the get_json() method of the api object called to send request 

ForAdmissionAndCases= AdmissionAndCases.get_json() 

#print(ForAdmissionAndCases) 



# saving dictioanry by using the json module to convert them into JSON format

import json
with open("ForAdmissionAndCases.json", "wt") as OUTF:
    json.dump(ForAdmissionAndCases, OUTF)


In [ ]:
# second implementation 
# same comments as above 


United_Kingdom = [
    'areaType=overview', 
] 

TestComparison = {
    "date": "date",
    "Test by Date":  "cumTestsByPublishDate", 
    "Pillar 1 Test": "cumPillarOneTestsByPublishDate",
    "Pillar 2 Test": "cumPillarTwoTestsByPublishDate",
    "Pillar 3 Test": "cumPillarThreeTestsByPublishDate", 
    "Pillar 4 Test" : "cumPillarFourTestsByPublishDate"}

TestComparison = Cov19API(filters=United_Kingdom, structure=TestComparison)

ForTestComparison = TestComparison.get_json()

#print(ForTestComparison)

import json
with open("ForTestComparison.json", "wt") as OUTF:
    json.dump(ForTestComparison, OUTF)

## Wrangling the data


In [ ]:
# first graph 



# opening Json file where data is saved 
with open("ForAdmissionAndCases.json", "rt") as INFILE:
    data1=json.load(INFILE)

    
AdmissionAndCasesDATA=data1['data']#
#AdmissionAndCasesDATA

# calling parameter dates1 and sorting the dates them

dates1=[dictionary['date'] for dictionary in AdmissionAndCasesDATA] 
dates1.sort() 
#dates1

# calling the data from cell above
def fill_in_date(dates1):
    return pd.to_datetime(dates1, format="%Y-%m-%d")


#sorting earliest date to lastest 
startdate=fill_in_date(dates1[0])
enddate=fill_in_date(dates1[-1])
print (startdate, ' to ', enddate) 


# defining dataFrame and creating an index as a date_range - includes any missing dates
index=pd.date_range(startdate, enddate, freq='D')
ForAdmissionAndCases=pd.DataFrame(index=index, columns=['New Admissions', 'Hospital Cases', 'New Cases', 
                                                        'Deaths within 28 days of a positive tests'
                                                       ])
#ForAdmissionAndCases 


# replacing missing data with zeros
     
for entry in AdmissionAndCasesDATA:
    date=fill_in_date(entry['date'])
    for column in ['New Admissions', 'Hospital Cases', 'New Cases']: 
        if pd.isna(ForAdmissionAndCases.loc[date, column]): 
            value= float(entry[column]) if entry[column]!=None else 0.0 
            ForAdmissionAndCases.loc[date, column]=value
        
ForAdmissionAndCases.fillna(0.0, inplace=True)

ForAdmissionAndCases.to_pickle("ForAdmissionAndCases.pkl")

In [ ]:
# graph 2 
# same comments as above 


# opening json file to write 
with open("ForTestComparison.json", "rt") as INFILE:
    data2=json.load(INFILE)
    
TestComparisonDATA=data2['data'] 
#TestComparisonDATA 

dates2=[dictionary['date'] for dictionary in TestComparisonDATA]
dates2.sort()
#dates2


def fill_in_date(dates2):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(dates2, format="%Y-%m-%d")

startdate=fill_in_date(dates2[0])
enddate=fill_in_date(dates2[-1])
print (startdate, ' to ', enddate)


index=pd.date_range(startdate, enddate, freq='D')
ForTestComparison=pd.DataFrame(index=index, columns=['Test by Date', 'Pillar 1 Test', 'Pillar 2 Test', 'Pillar 3 Test', 'Pillar 4 Test'
                                                       ]) 

#ForTestComparison 


for entry in TestComparisonDATA:
    date=fill_in_date(entry['date'])
    for column in ['Test by Date', 'Pillar 1 Test', 'Pillar 2 Test', 'Pillar 3 Test', 'Pillar 4 Test']:  
        if pd.isna(ForTestComparison.loc[date, column]): 
          value= float(entry[column]) if entry[column]!=None else 0.0 
        ForTestComparison.loc[date, column]=value
            
ForTestComparison.fillna(0.0, inplace=True)

# saving to a pickle file using pandas:
ForTestComparison.to_pickle("ForTestComparison.pkl")


## Graphs and Analysis

In [ ]:

# reading pickle files
ForAdmissionAndCases=pd.read_pickle("ForAdmissionAndCases.pkl")
ForTestComparison=pd.read_pickle("ForTestComparison.pkl")


In [ ]:

# API button for graph 1
def AdmissionAndCases_api(button): 
    apibutton.icon="check"
    apibutton.disabled=True  

apibutton=wdg.Button(
    description='Click to Download', 
    disabled=False, 
    button_style='success',  
    tooltip='Download Data',
    icon='download')


apibutton.on_click(AdmissionAndCases_api)
display(apibutton) 


series=wdg.SelectMultiple(
    options=['New Admissions', 'New Cases', 'Hospital Cases'],
    value=['New Admissions', 'New Cases', 'Hospital Cases'],
    rows=3,
    description='Stats',
    disabled=False)

scale=wdg.RadioButtons(
    options=['linear', 'log'],
    description='Scale',
    disabled=False)

controls=wdg.HBox([series, scale])


def AdmissionAndCases_graph(gcols, gscale):
    if gscale=='linear': 
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        ForAdmissionAndCases[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
    
    
graph=wdg.interactive_output(AdmissionAndCases_graph, {'gcols': series, 'gscale': scale})

display(controls, graph)    




In [ ]:

# API button for graph 2
def TestComparison_api(button): 
    apibutton1.icon="check"
    apibutton1.disabled=True  
    
apibutton1=wdg.Button(
description='Click to Download',
disabled=False,
button_style='info', 
tooltip='click to download data',
icon='download')

apibutton1.on_click(TestComparison_api)
display(apibutton1) 



testcols=wdg.SelectMultiple(
    options=['Pillar 1 Test', 'Pillar 2 Test', 'Pillar 3 Test', 'Pillar 4 Test'],
    value=['Pillar 1 Test', 'Pillar 2 Test', 'Pillar 3 Test', 'Pillar 4 Test'], # initial value
    rows=4, # rows of the selection box
    description='Tests:',
    disabled=False) 



def TestComparison_graph(graphcolumns):
        plt.show() 
    
output=wdg.interactive_output(TestComparison_graph, {'graphcolumns': testcols})

#display(testcols, output)

control=wdg.HBox([output, testcols, scale])

def TestComparison_graph(gcols, gscale):
    if gscale=='linear': 
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        ForTestComparison[list(gcols)].plot(logy=logscale)
        plt.show() 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
 
        
graph=wdg.interactive_output(TestComparison_graph, {'gcols': testcols, 'gscale': scale})



display(control, graph)


Covid-19 Dashboard (C) 17223-SS025, (2022). All rights reserved.